<a href="https://colab.research.google.com/github/SeaJungg/bigdata_analytics2021/blob/master/p%EB%B0%B8%EB%A5%98%EC%97%90%EB%AF%B8%EC%B3%90%EC%84%9C_%EB%8F%8C%EC%95%84%EC%98%A8_%EC%84%B8%EC%95%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from google.colab import files 


In [ ]:
uploaded = files.upload()
uploaded2 = files.upload()

Saving secom_labels.data to secom_labels.data


Saving secom.data to secom.data


In [ ]:
import io

In [ ]:
df = pd.read_csv(io.BytesIO(uploaded2['secom.data']), sep = ' ', header = None )
label = pd.read_csv(io.BytesIO(uploaded['secom_labels.data']), sep = ' ', header = None )[0]

In [ ]:
 col_candidate = []
#COl 기준 std가 0이면 제외하기
for idx, val in zip(df.columns, df.describe().iloc[2]):
  if val != 0:
    col_candidate.append(idx)
df = df[col_candidate]
print(len(col_candidate))

474


In [ ]:
#COl 기준 데이터의 수가 55% 이상인 것만 살리기
col_candidate = []
for idx, val in zip(df.columns, df.describe().iloc[0]):
  if val >= 700:
    col_candidate.append(idx)
df = df[col_candidate]
print(len(col_candidate))

450


In [ ]:
!pip install impyute

In [ ]:
import copy
data_with_label = copy.deepcopy(df)
data_with_label['Class'] = label

from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=3)
imputed = imputer.fit_transform(data_with_label)
data_with_label_imputed = pd.DataFrame(imputed, columns=data_with_label.columns)

In [ ]:
check_for_any_nan= data_with_label_imputed .isna().values.any()

In [ ]:
print("NaN Presence:"+str(check_for_any_nan))

NaN Presence:False


In [ ]:
from statsmodels.formula.api import ols

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
print(data_with_label_imputed.columns)

Index([      0,       1,       2,       3,       4,       6,       7,       8,
             9,      10,
       ...
           577,     582,     583,     584,     585,     586,     587,     588,
           589, 'Class'],
      dtype='object', length=451)


In [ ]:
#걍 시험삼아 첫번째 열에다가 회귀분석 실시.
test_col = data_with_label_imputed[0]
test_col = pd.concat([test_col,label],axis=1)
test_col.columns = ['test_col','Class']

In [ ]:
check_for_any_nan= test_col .isna().values.any()
print("NaN Presence:"+str(check_for_any_nan))

NaN Presence:True


In [ ]:
temp =[]
res = ols('Class~test_col', data=test_col).fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Class   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.9425
Date:                Tue, 01 Jun 2021   Prob (F-statistic):              0.332
Time:                        06:41:33   Log-Likelihood:                -1130.1
No. Observations:                1567   AIC:                             2264.
Df Residuals:                    1565   BIC:                             2275.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.3667      0.516     -0.711      0.477      -1.378       0.645
test_col      -0.0002      0.000     -0.971      0.332      -0.001       0.000
==============================================================================
Omnibus:                     1110.617   Durbin-Watson:                   1.771
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             9861.749
Skew:                           3.481   Prob(JB):                         0.00
Kurtosis:                      13.128   Cond. No.                     1.24e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.24e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [ ]:
res.tvalues

Intercept   -0.710872
test_col    -0.970829
dtype: float64

In [ ]:
res.pvalues[1]

0.0022796026874769572

In [ ]:
print(res.fvalue) #F-statistic
print(res.f_pvalue) #Prob (F-statistic)

0.9425082852447177
0.33178362551515383


In [ ]:
tvalue = []
fstatic = []
f_pvalue = []
pvalue = []

In [ ]:
for i in data_with_label_imputed.columns.values:
  print(i)
  test_col = data_with_label_imputed[i]
  test_col = pd.concat([test_col,label],axis=1)
  test_col.columns = ['test_col','Class']
  res = ols('Class~test_col', data=test_col).fit()
  pvalue.append([i,res.pvalues[1]])
  tvalue.append([i,res.tvalues])
  fstatic.append([i,res.fvalue])
  f_pvalue.append([i,res.f_pvalue])

0
1
2
3
4
6
7
8
9
10
11
12
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
43
44
45
46
47
48
50
51
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
86
87
88
89
90
91
92
93
94
95
96
98
99
100
101
102
103
104
105
106
107
108
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
142
143
144
145
146
147
148
150
151
152
153
154
155
156
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
180
181
182
183
184
185
187
188
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
221
222
223
224
225
227
228
238
239
247
248
249
250
251
252
253
254
255
267
268
269
270
271
272
273
274
275
277
278
279
280
281
282
283
285
286
287
288
289
290
291
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
316
317
318
319
320
321
323
324
331
332
333
334
335
336
337
338
339
340
341
3

In [ ]:
print(pvalue)

[[0, 0.33178362551491436], [1, 0.953401464396296], [2, 0.940510148375348], [3, 0.3348811816208863], [4, 0.589795494882699], [6, 0.512752825215812], [7, 0.5288148405428261], [8, 0.2662207237402001], [9, 0.21986391898144017], [10, 0.1808429481270553], [11, 0.19824764251474544], [12, 0.8167516779888429], [14, 0.006082694916188091], [15, 0.9068312943150256], [16, 0.924245792711067], [17, 0.7050896965588649], [18, 0.7228086673441424], [19, 0.22807964419642074], [20, 0.3576435435649833], [21, 2.2404024318791167e-05], [22, 0.004009767962503907], [23, 0.5951996723345029], [24, 0.4838749229079842], [25, 0.15126328562497324], [26, 0.0013479270100311058], [27, 0.24824326434391872], [28, 1.47155605625527e-05], [29, 0.4700302573410009], [30, 0.1336708345959043], [31, 0.05400845141532351], [32, 0.039190446338839606], [33, 0.0013359852538785333], [34, 0.9190839116987193], [35, 0.6280636550553269], [36, 0.9190810698295218], [37, 0.10132845190791054], [38, 0.03704885382695859], [39, 0.9358078184903029]